# Exploring one trip

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as pltb
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score
import seaborn as sns

import statsmodels.api as sm

from d2e2f.visualization import visualize
import re
import plotly.graph_objects as go

In [ ]:
loaded = catalog.load('trip_statistics_joined_thrusters')
df = pd.DataFrame()
for partition_id, loader in loaded.items():
    df_ = loader()
    df_['partition_id'] = partition_id
    df_['start_time'] = pd.to_datetime(df_['start_time'])
    df_['end_time'] = pd.to_datetime(df_['end_time'])
    
    df = df.append(df_, ignore_index=True)

df.sort_values(by='start_time', inplace=True)

In [ ]:
df['ship'] = df['partition_id'].apply(lambda x:re.search(r'([^-]+)',x).groups(1)[0])

In [ ]:
fig = px.scatter(df, x='start_time',y='P', color='ship', width=1000, height=600, 
                color_discrete_sequence=['red','green'], hover_data=['trip_no','trip_time'])
fig.show()

In [ ]:
dataset = catalog.load('data_with_trip_columns')
dataset.keys()

In [ ]:
loader = dataset['tycho-brahe-wind-2021-04-10--2021-06-30.parquet']
df_time = loader()

In [ ]:
trips = df_time.groupby('trip_no')

In [ ]:
trip = trips.get_group(556)

In [ ]:
trip['trip_direction'].dtype == 'object'

In [ ]:
visualize.plot_map(trip, time_step=None, color_key='trip_direction')

In [ ]:
trip.head()

In [ ]:
df_ = trip.copy()
df_['alpha'] = alpha = np.deg2rad(df_['wa'] - df_['heading'])  # True pointing angle relative to wind
W = df_['w']
V = df_['sog']
df_['aw_'] = A = np.sqrt(W**2 + V**2 + 2*W*V*np.cos(alpha))
df_['beta'] = np.arccos((W*np.cos(alpha) + V)/A)
df_['beta_deg'] = np.rad2deg(df_['beta'])

px.line(data_frame=df_.iloc[0:-1000], y=['awa','beta_deg'], width=900, height=300,)

In [ ]:
px.line(data_frame=df_.iloc[0:-1000], y=['aw','aw_'], width=900, height=300,)

In [ ]:
px.line(data_frame=df_, y=['wa','awa','heading'], width=900, height=300,)

In [ ]:
px.line(data_frame=trip, y=['w','aw','sog'], width=900, height=300,)

In [ ]:
px.line(data_frame=trip, y=['heading','cog'], width=900, height=300,)